# Calibration with moments
The average wage at occupation $j$ is $$\mathcal M^{theo}_{1j} =\frac{\int w_{ij}n_{ij}di}{\int n_{ij}di}.$$ 

We do not observe type $i$, but the occupation (current or previous). We define $u_{ij_0}$ the measure of unemployed workers previously employed in occupation $j_0$.
\begin{align}
    & u_{ij_0}=\frac{\delta_{j_0} n_{ij_0}}{\int s(e^0_{ij_1})m_{j_1}dj_1}
\end{align}
We can define i) the (instantaneous) probability of workers initially unemployed from occupation $j_0$ to obtain a job in occupation $j_1$, and ii) the probability of workers coming from occupation $j_0$ to obtain a job in occupation $j_1$.
\begin{align}
    & \mathcal M^{theo}_{2j_0j_1} =  \frac{\int s(e^0_{ij_1})m_{j_1}u_{ij_0}di}{\int u_{ij_0}di}\\
    & \mathcal M^{theo}_{3j_0j_1} = \frac{\int \xi s(e^{j_0}_{ij_1})m_{j_1}n_{ij_0}di}{\int n_{ij_0}di}\\
\end{align}
Mom 4?
$$\mathcal M^{theo}_{4j} = \int n_{ij}di$$

In [ ]:
function compute_uu(A::Alloc,M::Para,N::Hyper)
    uu = zeros(N.I,N.J)
    for i in 1:N.I
        denom = 0.
        for j1 in 1:N.J
            denom += s(A.e0[i,j1],M) * m(A.theta[j1],M)
        end
        
        for j0 in 1:N.J 
            uu[i,j0] = M.delta[j0] * A.n[i,j0] / denom  
        end
    end
    return(uu)
end

function theo_moments(A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
        
    uu = compute_uu(A,M,N)
    mom1 = zeros(N.J)
    mom2 = zeros(N.J,N.J)
    mom3 = zeros(N.J,N.J)
    mom4 = zeros(N.J)
    
    
    for j0 in 1:N.J
        uj0 = 0.
        for i in 1:N.I
            mom4[j0] += A.n[i,j0]
            uj0 += uu[i,j0]
            mom1[j0] += wage(i,j0,A,T,M,N,betatilde=betatilde) * A.n[i,j0]
        end
        mom1[j0] /= mom4[j0]
        
        for j1 in 1:N.J
            for i in 1:N.I
                mom2[j0,j1] += s(A.e0[i,j1],M) * m(A.theta[j1],M) * uu[i,j0]
                mom3[j0,j1] += M.xi * s(A.e1[i,j0,j1],M) * m(A.theta[j1],M) * A.n[i,j0]
            end
            uj0 = uj0>0 ? uj0 : 1e2 ## penality if uj0==0 
            mom2[j0,j1] /= uj0
            mom3[j0,j1] /= mom4[j0]
            #if j0==j1
              #  mom3[j0,j1] = 0. ## in the theory e1[j0,j0]=0 so mom3[j0,j0]=0
            #end
        end
    end
    
    return(Moments(mom1, mom2, mom3, mom4) )
end


function build_weights(emp::Moments)
    m1 = mean(emp.mom1)
    m2 = mean(emp.mom2)
    m4 = mean(emp.mom4)
    mycount = 0.
    mysum = 0.
    for j0 in 1:length(emp.mom1)
        for j1 in 1:(j0-1)
            mysum += emp.mom3[j0,j1] + emp.mom3[j1,j0]
            mycount += 2
        end
    end
    m3 = mysum/mycount
    return( [1/m1,1/m2,1/m3,1/m4] )
end  


function distance_mom(d1::Moments,d2::Moments, wgt, N::Hyper)
    totmom1 = sum(abs2,(d1.mom1-d2.mom1) * wgt[1]) / N.J
    totmom2 = sum(abs2,(d1.mom2-d2.mom2) * wgt[2]) / (N.J*N.J)
    totmom3 = 0
    for j0 in 1:N.J
        for j1 in 1:(j0-1)
            totmom3 += ((d1.mom3[j0,j1]-d2.mom3[j0,j1]) * wgt[3])^2
            totmom3 += ((d1.mom3[j1,j0]-d2.mom3[j1,j0]) * wgt[3])^2
        end
    end
    totmom3 /= N.J*(N.J-1) 
    totmom4 = sum(abs2,(d1.mom4-d2.mom4) * wgt[4]) / N.J
    return(totmom1+totmom2+totmom3+totmom4)
end
distance_mom(par::ParCalib, A,T,M, N::Hyper) = distance_mom(par.emp,theo_moments(A,T,M,N),par.wgt,N)

We take the parametric structure for $y_{ij}$ with $0\leq \alpha_j\leq 1$
$$
y_{ij}=(1-\alpha_j \mathbf{1}_{i\ne j})y_{jj}+\alpha_j \mathbf{1}_{i\ne j} h
$$

In [ ]:
function build_y(yhigh,alpha,h)
    y = zeros(length(yhigh),length(yhigh)) + Diagonal(yhigh)
    for j0 in 1:length(yhigh)
        for j1 in 1:(j0-1)
            y[j0,j1] = (1-alpha[j1]) * yhigh[j1] + alpha[j1] * h
            y[j1,j0] = (1-alpha[j0]) * yhigh[j0] + alpha[j0] * h
        end
    end
    return(y)
end

## 1) Calibrate a segmented allocation
We can calibrate a segmented allocation based on a subset of moments.
We look for $y_{jj}$ and vacancy costs $k_j$.
We avoid computing $\theta_j$ through a fixed-point algorithm by minimising the moment distance in $y_{jj}$ and $\theta_j$.
There are as many parameters as unknown, so we expect the distance to reach 0.


We choose an absolute tolerance: 0.1 for $y_{jj}$ and 0.001 for $\nu_i$ (we put a large number for $\theta$ so that it is not binding)

In [ ]:
function objective_calibrate_segm_equilibrium(vec,
        par::ParCalib,T::Taxes,M::Para,N::Hyper;
        betatilde=M.beta / (M.beta + (1-M.beta)*(1+T.w)) )
    ## vec = [ydiag,theta,nuc]
    
    (A1,M1) = compute_segm_equilibrium_theta(vec,T,M,N,betatilde=betatilde)
    out = distance_mom(par, A1,T,M1,N)
    return(out)
end


function calibrate_segm_equilibrium(par::ParCalib, T::Taxes, M::Para,N::Hyper; 
        betatilde=M.beta / (M.beta + (1-M.beta)*(1+T.w)),  
        toestim=[true,true,true], ## whether to estimate yd, theta and nu
        verbose= false, maxti=600, algo=:LN_SBPLX) 
    
    ## initialise vec = [ydiag,theta,nuc]
    ydiag0 = par.emp.mom1 * 3
    theta0 = fill(1e-5,N.J)
    nuc0 = nu2nuc(par.emp.mom4 ./ sum(par.emp.mom4 ))
    vec0 = vcat(ydiag0, theta0, nuc0)

    ## parameters for optimization
    opt = Opt(algo, length(vec0)) 
    maxtime!(opt,maxti)
    xtol_abs!(opt, vcat(fill(0.1,N.J), fill(1,N.J), fill(0.001/((N.I-1)*0.01),N.I-1 )))
    
    # bounds
    lbd = par.lbd
    ubd = par.ubd
    lbd_vec = vcat(fill(lbd.y,N.J), fill(lbd.theta,N.J), fill(lbd.nuc,N.I-1 ))
    ubd_vec = vcat(fill(ubd.y,N.J), fill(ubd.theta,N.J), fill(ubd.nuc,N.I-1) )
    toestim_vec = vcat(fill(toestim[1],N.J), fill(toestim[2],N.J), fill(toestim[3],N.I-1) )
    lower_bounds!(opt, toestim_vec .* lbd_vec + (1 .- toestim_vec) .* vec0 )
    upper_bounds!(opt, toestim_vec .* ubd_vec + (1 .- toestim_vec) .* vec0 )
    
    function obj(vec,g) 
        out = objective_calibrate_segm_equilibrium(vec,par,T,M,N,  betatilde=betatilde)
        verbose ? print(out," ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,vec,ret) = optimize(opt,vec0)
    println("The minimum is reached at ",minf, " with ", ret) 
  
    ## output
    (A1,M1) = compute_segm_equilibrium_theta(vec,T,M,N,betatilde=betatilde)    
    return(A1, M1)
    
end

## 2) Full calibration
We look for $y_{jj}$, $\alpha_j$, $\epsilon$, $\xi$ and $\theta_j$.
We then compute the $k_j$ that give the market tightness $\theta_j$ at equilibrium.

In [ ]:
function objective_calibrate_equilibrium(vec, par::ParCalib,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    Xtol=1e-2)
    ## vec=[yhigh,alpha,es,xi,theta,nuc]
    
    (A1,M1) = compute_equilibrium_theta(vec,T,M,N,betatilde=betatilde,Xtol=Xtol)
    out = distance_mom(par, A1,T,M1,N)    
    return(out)
end



function calibrate_equilibrium(par::ParCalib,T::Taxes,M::Para,N::Hyper ;
        betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
        A0 = compute_segm_equilibrium(T,M,N),
        toestim= fill(true,6), ##whether to estimate each of the 6: yd, alpha, es, xi, theta, nu
        Xtol=1e-2, verbose = false, maxti= 600, algo=:LN_SBPLX) # :LN_BOBYQA
         
    lbd = par.lbd
    ubd = par.ubd
    
    ## initialisation vec=[yhigh,alpha,es,xi,theta,nuc]
    alpha0 = vcat( (M.y[1,1]-M.y[2,1])/(M.y[1,1]-M.h), 
        [(M.y[i,i]-M.y[1,i])/(M.y[i,i]-M.h) for i in 2:N.J]  )
    vec0 = vcat(diag(M.y), min.(alpha0,ubd.alpha), M.es, M.xi, 
        max.(A0.theta,lbd.theta), nu2nuc(M.nu))

    ## parameters for optimization
    opt = Opt(algo, length(vec0)) 
    maxtime!(opt,maxti)
    
    lbd = par.lbd
    ubd = par.ubd
    lbd_vec = vcat(fill(lbd.y,N.J), fill(lbd.alpha,N.J), lbd.es, lbd.xi, 
                        fill(lbd.theta,N.J), fill(lbd.nuc,N.I-1 ) )
    ubd_vec = vcat(fill(ubd.y,N.J), fill(ubd.alpha,N.J), ubd.es, ubd.xi,
                        fill(ubd.theta,N.J), fill(ubd.nuc,N.I-1 ) )
    toestim_vec = vcat(fill(toestim[1],N.J), fill(toestim[2],N.J), 
        toestim[3], toestim[4], fill(toestim[5],N.J), fill(toestim[6],N.I-1) )
    lower_bounds!(opt, toestim_vec .* lbd_vec + (1 .- toestim_vec) .* vec0 )
    upper_bounds!(opt, toestim_vec .* ubd_vec + (1 .- toestim_vec) .* vec0 )
    
    function obj(vec,g) 
        #println("par: ",vec[N.J*2+1], " ", vec[N.J*2+2]," ", vec[N.J*2+3])
        out = objective_calibrate_equilibrium(vec,
            par,T,M,N,Xtol=Xtol,  betatilde=betatilde)
        verbose ? print(out," ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,vec,ret) = optimize(opt,vec0)
    println("The minimum is reached at ",minf, " with ", ret)

    ## output
    (A1,M1) = compute_equilibrium_theta(vec,T,M,N,betatilde=betatilde,Xtol=Xtol)
    return(A1, M1)
end


## calibrate with a fix alpha (uniform) and xi
function calibrate_equilibrium_step1(par::ParCalib,T::Taxes,M::Para,N::Hyper ;
        betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
        A0 = compute_segm_equilibrium(T,M,N),
        Xtol=1e-2, verbose = false, maxti= 600, algo=:LN_SBPLX) # :LN_BOBYQA
         
    ## initialisation vec=[alpha0,xi]
    alpha0 = (M.y[1,1]-M.y[2,1])/(M.y[1,1]-M.h)
    axi0 = [alpha0,M.xi]

    ## parameters for optimization
    opt = Opt(algo, 2) 
    maxtime!(opt,maxti)
    lower_bounds!(opt, [par.lbd.alpha, par.lbd.xi] )
    upper_bounds!(opt, [par.ubd.alpha, par.ubd.xi] )
    
    function obj(axi,g) 
        vec = vcat(diag(M.y), fill(axi[1],N.J), M.es, axi[2], 
                    max.(A0.theta,par.lbd.theta), nu2nuc(M.nu))
        out = objective_calibrate_equilibrium(vec,
                    par,T,M,N,Xtol=Xtol,  betatilde=betatilde)
        verbose ? print(out," ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,axi,ret) = optimize(opt,axi0)
    println("The minimum is reached at ",minf, " with ", ret)

    ## output
    vec = vcat(diag(M.y), fill(axi[1],N.J), M.es, axi[2], 
            max.(A0.theta,par.lbd.theta), nu2nuc(M.nu))
    (A1,M1) = compute_equilibrium_theta(vec,T,M,N,betatilde=betatilde,Xtol=Xtol)
    return(A1, M1)
end

We directly compute the vacancy costs defined by:
\begin{align*}
    & k_{j_1} = q(\theta_{j_1}) (1-\tilde\beta)\frac{num_{j_1}}{den_{j_1}}\\
    & num_{j_1}=\int_{\mathcal I\times\mathcal J}\left( X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1}\right)\xi s(e_{ij_1}^{j_0})n_{ij_0}didj_0
     +\int_\mathcal{I}\left(X_{ij_1}-T_{ij_1}\right) s(e_{ij_1}^{0}) u_idi\\
     & den_{j_1}=\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})n_{ij_0}didj_0
     +\int_\mathcal{I}s(e_{ij_1}^{0}) u_idi
\end{align*}

In [ ]:
function compute_k(A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))

    k = zeros(N.J)
    for j1 in 1:N.J
        den = 0.
        num = 0.
        for i in 1:N.I
            ## p = 0
            num +=  (A.X[i,j1] - T_tax(i,j1,A.X,T,M)) * s(A.e0[i,j1],M) * A.u[i]
            den += s(A.e0[i,j1],M) * A.u[i]
            
            ## p =j0
            for j0 in 1:N.J
                num +=  (A.X[i,j1] - A.X[i,j0] - TT_tax(i,j0,j1,A.X,T,M)) * M.xi * s(A.e1[i,j0,j1],M) * A.n[i,j0]
                den += M.xi * s(A.e1[i,j0,j1],M) * A.n[i,j0]
            end
        end
        k[j1] = q(A.theta[j1],M) * (1-betatilde) * num/den
    end 
    
    return(k)
end
